# HW 3  Что в векторе твоем?

## Получение датасета 

In [1]:
import requests      # Библиотека для отправки запросов
import numpy as np   # Библиотека для матриц, векторов и линала
import pandas as pd  # Библиотека для табличек
import time          # Библиотека для времени

import warnings
warnings.filterwarnings("ignore")


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import json
from tqdm import tqdm
from sklearn.metrics import *

Загружаем данные из ДЗ-1 Парсинг данных (https://github.com/korolevt/otus_NLP_2023/blob/main/hw1/hw1.ipynb)
Которая содержит информацию с комментариями к фильмам, с оценками

In [3]:
df = pd.read_pickle("data.pkl")
df

,Comment,Rating
0,"""Перестань волноваться о том, что не можешь ко...",5
1,"Двое пассажиров космического корабля, просыпаю...",4
2,"Давно не видела таких бредовых фильмов. ""Пасса...",1
3,"Человеку нужен человек. Если нельзя, но очень ...",5
4,Неплохая вариация на тему космоса и одиночества,4
...,...,...
6742,"Фильм, который стоит посмотреть!",5
6743,Завораживающий - breathtaking!,5
6744,"Малефисента-2 - сиквел удался, или мои впечатл...",4
6745,Вау!,5


Преобразуем dataframe в json

In [4]:
data = df.to_json(orient='records')
data = json.loads(data)

In [5]:
data[0]

{'Comment': '"Перестань волноваться о том, что не можешь контролировать, просто живи" - пожалуй, это самый лучший жизненный совет|Робинзонада в космосе✨🌙|Один из моих любимых фильмов. Когда мне грустно, я включаю "Пассажиры".',
 'Rating': 5}

In [6]:
from collections import Counter
print(len(data))
Counter([x['Rating'] for x in data]).most_common()

6747


[(5, 4469), (4, 1066), (3, 609), (2, 329), (1, 274)]

### Предобработка данных

Удаляем стоп-слова, пунктуацию

In [7]:
import nltk   # Natural Language Toolkit
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('russian')

word_tokenizer = nltk.WordPunctTokenizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tima-\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
import re

regex = re.compile(r'[А-Яа-яA-zёЁ]+')

import string
spec_chars = string.punctuation + '\d\n\xa0«»\t—…' 
regex2 = re.compile('[' + spec_chars + ']')

#Убираем пунктуацию и все остальное
def words_only(text):
    try:
        return " ".join(regex.findall(text)).lower()
    except:
        return ""

#Убираем пунктуацию, но оставляем смайлики
def words_only2(text):
    try:
        return re.sub(regex2, '', text.lower())
    except:
        return ""

def process_data(data):
    texts = []
    targets = []

    # поочередно проходим по всем новостям в списке
    for item in tqdm(data):

        text_lower = words_only2(item['Comment']) # оставим только слова
        tokens     = word_tokenizer.tokenize(text_lower) #разбиваем текст на слова

        # удаляем пунктуацию и стоп-слова
        tokens = [word for word in tokens if (word not in stop_words and not word.isnumeric())]

        texts.append(tokens) # добавляем в предобработанный список

    return texts

In [9]:
spec_chars

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~\\d\n\xa0«»\t—…'

In [10]:
y = [item['Rating'] for item in data]
texts = process_data(data)

100%|███████████████████████████████████████████████████████████████████████████| 6747/6747 [00:00<00:00, 36452.57it/s]


In [11]:
# example
i = 0
print("Rating: ", y[i])
print("Comment: ", texts[i])

Rating:  5
Comment:  ['перестань', 'волноваться', 'можешь', 'контролировать', 'просто', 'живи', 'пожалуй', 'это', 'самый', 'лучший', 'жизненный', 'советробинзонада', 'космосе', '✨🌙', 'моих', 'любимых', 'фильмов', 'грустно', 'включаю', 'пассажиры']


### Нормализация слов

In [12]:
#! pip install pymorphy3

In [13]:
# загружаем библиотеку для лемматизации
import pymorphy3 # Морфологический анализатор

# инициализируем лемматизатор :)
morph = pymorphy3.MorphAnalyzer()

In [14]:
i = 0
for aword in texts[i]:
    aword_norm = morph.parse(aword)[0].normal_form
    print("Исходное слово: %s\tЛемматизированное: %s" % (aword, aword_norm))

Исходное слово: перестань	Лемматизированное: перестать
Исходное слово: волноваться	Лемматизированное: волноваться
Исходное слово: можешь	Лемматизированное: мочь
Исходное слово: контролировать	Лемматизированное: контролировать
Исходное слово: просто	Лемматизированное: просто
Исходное слово: живи	Лемматизированное: живить
Исходное слово: пожалуй	Лемматизированное: пожалуй
Исходное слово: это	Лемматизированное: это
Исходное слово: самый	Лемматизированное: самый
Исходное слово: лучший	Лемматизированное: хороший
Исходное слово: жизненный	Лемматизированное: жизненный
Исходное слово: советробинзонада	Лемматизированное: советробинзонада
Исходное слово: космосе	Лемматизированное: космос
Исходное слово: ✨🌙	Лемматизированное: ✨🌙
Исходное слово: моих	Лемматизированное: мой
Исходное слово: любимых	Лемматизированное: любимый
Исходное слово: фильмов	Лемматизированное: фильм
Исходное слово: грустно	Лемматизированное: грустно
Исходное слово: включаю	Лемматизированное: включать
Исходное слово: пассажиры

In [15]:
# применяем лемматизацию ко всем текстам
texts_normal = texts.copy()
for i in range(len(texts)):           # tqdm_notebook создает шкалу прогресса :)
    text_lemmatized = [morph.parse(x)[0].normal_form for x in texts[i]] # применяем лемматизацию для каждого слова в тексте
    texts_normal[i] = ' '.join(text_lemmatized)                # объединяем все слова в одну строку через пробел

In [16]:
# example
i = 0
print("Rating: ", y[i])
print("Comment: ", texts_normal[i])

Rating:  5
Comment:  перестать волноваться мочь контролировать просто живить пожалуй это самый хороший жизненный советробинзонада космос ✨🌙 мой любимый фильм грустно включать пассажир


<b>Преобразуем целевую переменную</b>

```
Где оценка 1-3 - это негативные отзывы
Где оценка 4-5 - это положительные отзывы
```

In [17]:
# Функция для кодирования лейблов
def num2boolean(y):
    if y == 1 or y == 2 or y == 3:
        return 0
    if y == 4 or y == 5:
        return 1

y = [num2boolean(yy) for yy in y]

In [18]:
Counter(y)

Counter({1: 5535, 0: 1212})

Разбиваем данные

In [19]:
#train test_split
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_y, test_y = train_test_split(texts_normal, y, test_size=0.20, random_state=42, stratify = y)

### TF-IDF

In [48]:
#вычисляем tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
# Fit TF-IDF on train texts
#vectorizer = TfidfVectorizer(max_features = 1000, norm = None) # возмем топ 1000 слов
vectorizer = TfidfVectorizer( ngram_range=(1,2), min_df=2)
vectorizer.fit(train_texts)


TfidfVectorizer(min_df=2, ngram_range=(1, 2))

In [49]:
# Обучаем TF-IDF на train, а затем применяем к train и test
train_X = vectorizer.fit_transform(train_texts)
test_X  = vectorizer.transform(test_texts)

In [50]:
# Получение списка ключевых слов
tfidf_feature_names = np.array(vectorizer.get_feature_names_out())
tfidf_sorting = np.argsort(np.asarray(train_X.sum(axis=0)).ravel())[::-1]
n = 10
tfidf_feature_names[tfidf_sorting[:n]]

array(['фильм', 'хороший', 'смотреть', 'очень', 'хороший фильм', 'это',
       'шедевр', 'который', 'просмотр', 'слеза'], dtype=object)

In [51]:
def display_scores(vectorizer, tfidf_result):
    # http://stackoverflow.com/questions/16078015/
    scores = zip(vectorizer.get_feature_names_out(),
                 np.asarray(tfidf_result.sum(axis=0)).ravel())
    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
    for item in sorted_scores[:20]:
        print ("{0:50} Score: {1}".format(item[0], item[1]))
display_scores(vectorizer, train_X)

фильм                                              Score: 298.6678111467953
хороший                                            Score: 138.18984580969186
смотреть                                           Score: 91.329489526741
очень                                              Score: 81.99564750674608
хороший фильм                                      Score: 75.90427594535214
это                                                Score: 75.76494187710762
шедевр                                             Score: 65.78961846216994
который                                            Score: 62.77254715627877
просмотр                                           Score: 56.62598978848509
слеза                                              Score: 55.506075571525194
понравиться                                        Score: 51.889224451897704
время                                              Score: 51.664639281614214
посмотреть                                         Score: 51.479607007589216
жизнь    

In [34]:
# Здесь это не потребуется, так как у нас только один лайбел для классификации - положительный/отрицательный отзыв
#from sklearn.preprocessing import LabelEncoder
#le = LabelEncoder()
#le.fit(train_y);

###  RandomForest (Случайный лес)

In [145]:
%%time
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

model = RandomForestClassifier(random_state=42)
params = {
    'n_estimators': [500, 1000, 1500],
    'max_depth': [10,25,50]
}

clf = GridSearchCV(model, params, scoring='f1', verbose=10)
clf.fit(train_X, train_y)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START max_depth=10, n_estimators=500..............................
[CV 1/5; 1/9] END max_depth=10, n_estimators=500;, score=0.901 total time=   3.5s
[CV 2/5; 1/9] START max_depth=10, n_estimators=500..............................
[CV 2/5; 1/9] END max_depth=10, n_estimators=500;, score=0.901 total time=   3.6s
[CV 3/5; 1/9] START max_depth=10, n_estimators=500..............................
[CV 3/5; 1/9] END max_depth=10, n_estimators=500;, score=0.902 total time=   3.4s
[CV 4/5; 1/9] START max_depth=10, n_estimators=500..............................
[CV 4/5; 1/9] END max_depth=10, n_estimators=500;, score=0.901 total time=   4.0s
[CV 5/5; 1/9] START max_depth=10, n_estimators=500..............................
[CV 5/5; 1/9] END max_depth=10, n_estimators=500;, score=0.901 total time=   3.7s
[CV 1/5; 2/9] START max_depth=10, n_estimators=1000.............................
[CV 1/5; 2/9] END max_depth=10, n_estimators

GridSearchCV(estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [10, 25, 50],
                         'n_estimators': [500, 1000, 1500]},
             scoring='f1', verbose=10)

In [146]:
clf.best_params_

{'max_depth': 50, 'n_estimators': 500}

In [171]:
#import алгоритма из библиотеки
from sklearn.ensemble import RandomForestClassifier

# инициализируем модель
clf = RandomForestClassifier(n_estimators = 500, max_depth = 50, random_state=42)

# обучаем ее на тренировочных данных
clf = clf.fit(train_X, train_y)

# делаем предсказание для тестовых данных
pred = clf.predict(test_X)
pred_proba = clf.predict_proba(test_X)

In [172]:
print('Accuracy: ', accuracy_score(test_y, pred))
print('F1: ', f1_score(test_y, pred, average = 'macro'))

Accuracy:  0.8281481481481482
F1:  0.5063051702395965


In [173]:
print(classification_report(test_y, pred))

              precision    recall  f1-score   support

           0       0.82      0.06      0.11       243
           1       0.83      1.00      0.90      1107

    accuracy                           0.83      1350
   macro avg       0.83      0.53      0.51      1350
weighted avg       0.83      0.83      0.76      1350



In [174]:
roc_auc_score(test_y, pred_proba[:,1], average='macro')

0.7806662428764206

### LGDB (Cтохастического градиентый бустинг)

In [141]:
#!pip install lightgbm

In [ ]:
%%time
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

model = LGBMClassifier(random_state=42)
params = {
    'num_leaves': [5, 15, 31], # default=31
    'max_depth': [-1, 3, 7], # default=-1
    'learning_rate': [0.5, 0.1, 0.01], # default=0.1
}

clf = GridSearchCV(model, params, scoring='f1', verbose=10)
#clf.fit(train_X, le.transform(train_y))
clf.fit(train_X, train_y)

In [53]:
clf.best_params_

{'learning_rate': 0.1, 'max_depth': 7, 'num_leaves': 31}

In [ ]:
# saving time after GridSearch
from lightgbm import LGBMClassifier

clf = LGBMClassifier(**{'learning_rate': 0.1, 'max_depth': 7, 'num_leaves': 31}, random_state=42)
clf.fit(train_X, train_y);

# делаем предсказание для тестовых данных
pred = clf.predict(test_X)
pred_proba = clf.predict_proba(test_X)

In [55]:
print('Accuracy: ', accuracy_score(test_y, pred))
print('F1: ', f1_score(test_y, pred, average = 'macro'))

Accuracy:  0.8251851851851851
F1:  0.4943146105936804


In [56]:
print(classification_report(test_y, pred))

              precision    recall  f1-score   support

           0       0.73      0.05      0.09       243
           1       0.83      1.00      0.90      1107

    accuracy                           0.83      1350
   macro avg       0.78      0.52      0.49      1350
weighted avg       0.81      0.83      0.76      1350



In [169]:
roc_auc_score(le.transform(test_y), pred_proba[:,1], average='macro')

0.7786588153947382

### FastText emb + LGBM

In [102]:
#!pip install fasttext
#!pip install fasttext-0.9.2-cp311-cp311-win_amd64.whl

In [60]:
import fasttext
from huggingface_hub import hf_hub_download

In [58]:
help(fasttext)

Help on package fasttext:

NAME
    fasttext

DESCRIPTION
    # Copyright (c) 2017-present, Facebook, Inc.
    # All rights reserved.
    #
    # This source code is licensed under the MIT license found in the
    # LICENSE file in the root directory of this source tree.

PACKAGE CONTENTS
    FastText
    tests (package)
    util (package)

DATA
    BOW = '<'
    EOS = '</s>'
    EOW = '>'

FILE
    d:\python\anaconda3\lib\site-packages\fasttext\__init__.py




In [61]:
# Заграужаем модель с уменьшенной размерностью (до 100) от facebook
model_path = hf_hub_download(repo_id="kernela/fasttext-ru-vectors-dim-100", filename="ru-vectors-dim-100.bin")
model_fasttext = fasttext.load_model(model_path)

In [60]:
len(model_fasttext.words)

2000000

In [65]:
def cosine_similarity(word1, word2):
    return np.dot(model_fasttext[word1], model_fasttext[word2]) / (np.linalg.norm(model_fasttext[word1]) * np.linalg.norm(model_fasttext[word2]))

In [66]:
print(cosine_similarity("мужчина", "мальчик"))

0.8241205


In [28]:
df = pd.DataFrame([texts_normal, y]).transpose()
df.columns=['text_normal', 'y']
df

,text_normal,y
0,перестать волноваться мочь контролировать прос...,1
1,двое пассажир космический корабль просыпаться ...,1
2,давно видеть такой бредовый фильм пассажир отз...,0
3,человек нужный человек очень хотеться очень пр...,1
4,неплохой вариация тема космос одиночество,1
...,...,...
6742,фильм который стоить посмотреть,1
6743,завораживать breathtaking,1
6744,малефисент сиквел удаться мой впечатление прос...,1
6745,вау,1


In [53]:
# Сохраняем данные с номрмализацией комментариев
pd.to_pickle(df, "data_norm.pkl")

In [29]:
from tqdm.notebook import tqdm
tqdm.pandas()

embeddings = df['text_normal'].progress_apply(model_fasttext.get_sentence_vector)

  0%|          | 0/6747 [00:00<?, ?it/s]

In [30]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(np.array(embeddings.tolist()), y, test_size=0.20, random_state=42, stratify = y)

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

model = LGBMClassifier(random_state=42)
params = {
    'num_leaves': [5, 15, 31], # default=31
    'max_depth': [-1, 3, 7], # default=-1
    'learning_rate': [0.5, 0.1, 0.01], # default=0.1
}

clf = GridSearchCV(model, params, scoring='f1', verbose=10)
clf.fit(train_X, train_y)

In [74]:
clf.best_params_

{'learning_rate': 0.1, 'max_depth': 3, 'num_leaves': 5}

In [75]:
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

clf = LGBMClassifier(**{'learning_rate': 0.1, 'max_depth': 3, 'num_leaves': 5}, random_state=42)
clf.fit(train_X, train_y);

# делаем предсказание для тестовых данных
pred = clf.predict(test_X)
pred_proba = clf.predict_proba(test_X)

[LightGBM] [Info] Number of positive: 4428, number of negative: 969
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002563 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25498
[LightGBM] [Info] Number of data points in the train set: 5397, number of used features: 100
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.820456 -> initscore=1.519439
[LightGBM] [Info] Start training from score 1.519439


In [76]:
print('Accuracy: ', accuracy_score(test_y, pred))
print('F1: ', f1_score(test_y, pred, average = 'macro'))

Accuracy:  0.8296296296296296
F1:  0.5557304335384176


In [78]:
roc_auc_score(test_y, pred_proba[:,1], average='macro')

0.8005100352786793

### FastText emb + TF-IDF

In [57]:
train_texts, test_texts, train_y, test_y = train_test_split(texts_normal, y, test_size=0.20, random_state=42, stratify = y)

In [62]:
vectorizer = TfidfVectorizer(vocabulary=model_fasttext.get_words(), ngram_range=(1,2), min_df=2)
vectorizer.fit(train_texts)

TfidfVectorizer(min_df=2, ngram_range=(1, 2),
                vocabulary=[',', '.', 'и', 'в', '</s>', ':', ')', '(', 'на',
                            '"', 'с', 'не', '»', 'для', '-', '«', '/', 'по',
                            '—', 'что', 'В', '!', 'из', 'от', 'к', 'как', '?',
                            'а', '–', 'за', ...])

In [63]:
# Обучаем TF-IDF на train, а затем применяем к train и test
train_X_tfidf = vectorizer.fit_transform(train_texts)
test_X_tfidf  = vectorizer.transform(test_texts)

In [64]:
display_scores(vectorizer, train_X_tfidf)

фильм                                              Score: 383.1926263860503
хороший                                            Score: 178.44584044817157
смотреть                                           Score: 115.66711741641754
очень                                              Score: 111.56906994847239
это                                                Score: 92.38582773668638
который                                            Score: 78.88806054745073
просмотр                                           Score: 75.12107809807551
шедевр                                             Score: 74.6171813299039
слеза                                              Score: 64.37976826727315
посмотреть                                         Score: 63.047771785893744
жизнь                                              Score: 61.830475832528485
понравиться                                        Score: 59.86405620918947
время                                              Score: 59.43778057654602
замечате

In [65]:
vocab = vectorizer.get_feature_names_out()
len(vocab)

2000000

In [66]:
# Подсчет эмбединга fasttext c весами tf-idf
from tqdm import tqdm

def get_emb_with_tfidf(sents, vectorized):
    embeddings = []
    with tqdm(total=vectorized.shape[0]) as pbar:
        for n, row in enumerate(vectorized):
            _, idx = row.nonzero()
            tfidf_weights = row[0, idx].toarray()[0]
            emb = np.zeros(model_fasttext.get_dimension())  
            for weight, i in zip(tfidf_weights, idx):
                emb_fasttext = model_fasttext.get_word_vector(vocab[i])
                emb += weight * emb_fasttext
            embeddings.append(emb/len(idx))  # среднее значение
            pbar.update()
        return np.array(embeddings)

In [67]:
train_X = get_emb_with_tfidf(train_texts, train_X_tfidf)

100%|█████████████████████████████████████████████████████████████████████████████| 5397/5397 [00:48<00:00, 111.91it/s]


In [68]:
test_X = get_emb_with_tfidf(test_texts, test_X_tfidf)

100%|█████████████████████████████████████████████████████████████████████████████| 1350/1350 [00:11<00:00, 112.74it/s]


In [ ]:
model = LGBMClassifier(random_state=42)
params = {
    'num_leaves': [5, 15, 31], # default=31
    'max_depth': [-1, 3, 7], # default=-1
    'learning_rate': [0.5, 0.1, 0.01], # default=0.1
}

clf = GridSearchCV(model, params, scoring='f1', verbose=10)
clf.fit(train_X, train_y)

In [268]:
clf.best_params_

{'learning_rate': 0.1, 'max_depth': -1, 'num_leaves': 31}

In [269]:
# делаем предсказание для тестовых данных
pred = clf.predict(test_X)
pred_proba = clf.predict_proba(test_X)

In [270]:
print('Accuracy: ', accuracy_score(test_y, pred))
print('F1: ', f1_score(test_y, pred, average = 'macro'))

Accuracy:  0.8333333333333334
F1:  0.6022439334834002


In [271]:
print(classification_report(test_y, pred))

              precision    recall  f1-score   support

           0       0.62      0.20      0.30       243
           1       0.85      0.97      0.91      1107

    accuracy                           0.83      1350
   macro avg       0.73      0.59      0.60      1350
weighted avg       0.81      0.83      0.80      1350



In [272]:
roc_auc_score(test_y, pred_proba[:,1], average='macro')

0.7919951970438771

### Выводы

Были рассмотрены следующие классификаторы

Точность:

    TF_IDF + RandomForest:         Accuracy:  0.828 F1:  0.506
    TF_IDF + LGDB:                 Accuracy:  0.825 F1:  0.49
    FastText emb + LGBM:           Accuracy:  0.830 F1:  0.556
    FastText emb + TF-IDF + LGBM:  Accuracy:  0.833 F1:  0.602

Гиперпараметры подбирались с помощью GridSearch  
TF-IDF хорошо использовать для классификации  
Использование отдельно FastText эмбеддинг не сильно увеличивает точность по сравнению c TF-IDF  
Комбинация FastText эмбеддинга с весами TF-IDF и градиентным бустингом дали лучшую точность и увеличение F1 для несбалансированных классов. 
